In [ ]:
import pandas as pd
from tqdm import *

In [ ]:
!cp ../input/2021-vin-2class/sample_submission.csv sample_submission.csv

In [ ]:
pd.read_csv('sample_submission.csv').head()

,image_id,width,height,class14prob
0,002a34c58c5b758217ed1f584ccbcfe9,2345,2584,0.986674
1,004f33259ee4aef671c2b95d54e4be68,2517,3028,0.962765
2,008bdde2af2462e86fd373a445d0f4cd,2304,2880,0.060300
3,009bc039326338823ca3aa84381f17f1,1994,2430,0.876201
4,00a2145de1886cb9eb88869c85d74080,2136,2408,0.345994


In [ ]:
df = pd.DataFrame()
df['image_id'] = ['002a34c58c5b758217ed1f584ccbcfe9',
                  '002a34c58c5b758217ed1f584ccbcfe9',
                  '004f33259ee4aef671c2b95d54e4be68',
                  '008bdde2af2462e86fd373a445d0f4cd',
                  '009bc039326338823ca3aa84381f17f1']

df['class_id'] = ['0',
                  '1',
                  '2',
                  '3',
                  '14']

df['x_min'] = ['0.1',
               '0.2',
               '0.3',
               '0.4',
               '0']

df['x_max'] = ['0.2',
               '0.3',
               '0.4',
               '0.5',
               '1']

df['y_min'] = ['0.15',
               '0.25',
               '0.35',
               '0.45',
               '0']

df['y_max'] = ['0.25',
               '0.35',
               '0.45',
               '0.55',
               '1']

df['score'] = ['0.6',
               '0.7',
               '0.8',
               '0.9',
               '1']

df.head()

,image_id,class_id,x_min,x_max,y_min,y_max,score
0,002a34c58c5b758217ed1f584ccbcfe9,0,0.1,0.2,0.15,0.25,0.6
1,002a34c58c5b758217ed1f584ccbcfe9,1,0.2,0.3,0.25,0.35,0.7
2,004f33259ee4aef671c2b95d54e4be68,2,0.3,0.4,0.35,0.45,0.8
3,008bdde2af2462e86fd373a445d0f4cd,3,0.4,0.5,0.45,0.55,0.9
4,009bc039326338823ca3aa84381f17f1,14,0,1,0,1,1


In [ ]:
import pandas as pd
from tqdm import *
def to_submission(df, sample_file='sample_submission.csv', filename='submission.csv', trick=True, keep_ratio=False):
    '''
    df: output dataframe
    sample_file: sample_submission file (to get image_id)
    filename: to csv file name
    trick: add class 14 to all images
    
    return a dataframe
    '''
    
    ss = pd.read_csv(sample_file)
    d = {i:[] for i in ss['image_id'].values.tolist()}
    sizemap = {i:(w,h) for i,w,h in zip(ss['image_id'].values.tolist(), ss['width'].values.tolist(), ss['height'].values.tolist())}
    trick_string = {i:'14 '+str(s)+' 0 0 1 1' for i,s in zip(ss['image_id'].values.tolist(), ss['class14prob'].values.tolist())}
    
    image_id = []
    PredictionString = []
    for i in trange(len(df['image_id'].values.tolist())):
        w,h = sizemap[df['image_id'][i]]

        x_min = round(float(df['x_min'][i])*w)
        y_min = round(float(df['y_min'][i])*h)
        x_max = round(float(df['x_max'][i])*w)
        y_max = round(float(df['y_max'][i])*h)

        if (keep_ratio):
            if (w > h):
                y_min -= (w-h)/2
                y_max -= (w-h)/2
            else:
                x_min -= (h-w)/2
                x_max -= (h-w)/2
            

        d[df['image_id'][i]].append(" ".join(map(str, [int(df['class_id'][i]), df['score'][i], x_min, y_min, x_max, y_max] )))
        
    for (k,v) in d.items():
        image_id.append(k)
        if (len(v) == 0): #case 14 1 0 0 1 1 not in df
            PredictionString.append(trick_string[k])
        elif (len(v) == 1 and v[0].startswith("14 ")): #case 14 x 0 0 1 1 in df
            PredictionString.append(" ".join(v[0].split()[:2])+' 0 0 1 1')
        else:
            if (trick):
                v.append(trick_string[k])
            PredictionString.append(" ".join(v))
    result = pd.DataFrame()
    result['image_id'] = image_id
    result['PredictionString'] = PredictionString
    
    result.sort_values('image_id', ignore_index=True,  inplace=True)
    
    if (filename):
        result.to_csv(filename, index = False)
    return result

In [ ]:
submission = to_submission(df, trick = False)
submission.head(10)

100%|██████████| 5/5 [00:00<00:00, 5071.71it/s]


,image_id,PredictionString
0,002a34c58c5b758217ed1f584ccbcfe9,0 0.6 234 388 469 646 1 0.7 469 646 704 904
1,004f33259ee4aef671c2b95d54e4be68,2 0.8 755 1060 1007 1363
2,008bdde2af2462e86fd373a445d0f4cd,3 0.9 922 1296 1152 1584
3,009bc039326338823ca3aa84381f17f1,14 1 0 0 1 1
4,00a2145de1886cb9eb88869c85d74080,14 0.345993519 0 0 1 1
5,00b7e6bfa4dc1fe9ddd0ce74743e38c2,14 0.198367283 0 0 1 1
6,011295e0bcdc7636569ab73bfdcc4450,14 0.986467935 0 0 1 1
7,013c169f9dad6f1f6485da961b9f7bf2,14 0.362230472 0 0 1 1
8,01431a2618c0ace741e4e270a37e20b9,14 0.053823829 0 0 1 1
9,0168eb925aa6f28a78b16134792f5d0e,14 0.982547793 0 0 1 1


In [ ]:
submission = to_submission(df, filename = 'submission.csv')
submission.head(10)

100%|██████████| 5/5 [00:00<00:00, 9391.63it/s]


,image_id,PredictionString
0,002a34c58c5b758217ed1f584ccbcfe9,0 0.6 234 388 469 646 1 0.7 469 646 704 904 14...
1,004f33259ee4aef671c2b95d54e4be68,2 0.8 755 1060 1007 1363 14 0.962765466 0 0 1 1
2,008bdde2af2462e86fd373a445d0f4cd,3 0.9 922 1296 1152 1584 14 0.06030032 0 0 1 1
3,009bc039326338823ca3aa84381f17f1,14 1 0 0 1 1
4,00a2145de1886cb9eb88869c85d74080,14 0.345993519 0 0 1 1
5,00b7e6bfa4dc1fe9ddd0ce74743e38c2,14 0.198367283 0 0 1 1
6,011295e0bcdc7636569ab73bfdcc4450,14 0.986467935 0 0 1 1
7,013c169f9dad6f1f6485da961b9f7bf2,14 0.362230472 0 0 1 1
8,01431a2618c0ace741e4e270a37e20b9,14 0.053823829 0 0 1 1
9,0168eb925aa6f28a78b16134792f5d0e,14 0.982547793 0 0 1 1
